In [1]:
%cd /home/wzt/src/moco-v3-3d/

/home/wzt/src/moco-v3-3d


In [2]:
import transforms as my_transforms
import datasets as my_datasets
import monai
import torch

In [3]:
def make_transform(patch_size):
    aug_block = my_transforms.aug_transform(size=patch_size)
    return monai.transforms.Compose([
        monai.transforms.EnsureChannelFirst(channel_dim='no_channel'),
        my_transforms.Duplicate(transforms1=aug_block, transforms2=aug_block),
    ])

traindir = '/home/wzt/src/fmcibx/data/ROI_resize48/images_cut_48_resize'
train_dataset = my_datasets.SSLNoCropDataset(traindir, size=48, transform=make_transform(48), enable_negatives=False)
x, _ = train_dataset[0]
x1, x2 = x
print(x1.shape, x2.shape)

train_sampler = None if 1 else torch.utils.data.distributed.DistributedSampler(train_dataset)
train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=4, shuffle=(train_sampler is None),
    num_workers=0, pin_memory=True, sampler=train_sampler, drop_last=True)
print("train_loader: [OK]")

torch.Size([1, 48, 48, 48]) torch.Size([1, 48, 48, 48])
train_loader: [OK]


In [4]:
for i, (images, _) in enumerate(train_loader):
    break

In [5]:
from functools import partial
import moco.builder
import vits_3d
import json
from easydict import EasyDict as edict

with open("/home/wzt/src/moco-v3-3d/args.json") as f:
    args = edict(json.load(f))


model = moco.builder.MoCo_ViT(
    partial(vits_3d.__dict__[args.arch], stop_grad_conv1=args.stop_grad_conv1, img_size=48, in_chans=1),
    args.moco_dim, args.moco_mlp_dim, args.moco_t)


moco_m = args.moco_m

model(images[0], images[1], moco_m)

metatensor(1.3490, grad_fn=<AliasBackward0>)

In [6]:
model(torch.zeros_like(images[0]), torch.zeros_like(images[0]), moco_m)

metatensor(1.1090, grad_fn=<AliasBackward0>)

In [9]:
images[0].isnan().any() or images[1].isnan().any()

metatensor(False)